# Notebook 2: Manipulating Assets

This notebook you will program the robot to spell out your initials!. The structure is quite similar to the pick and place demo in [chapter 3](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/65aad364-ef1c-45f5-a796-fac7c122e274/notebook/pick-d0b41e97e5124292af7ed01072ecece4) of Robotic Manipulation. Take a moment to make sure that notebook makes sense to you.

Let's start by getting our imports out of the way and launching Meshcat. 

In [1]:
from pathlib import Path

import mpld3
import numpy as np
from matplotlib import pyplot as plt
from pydrake.all import (
    AddFrameTriadIllustration,
    AddMultibodyPlantSceneGraph,
    AngleAxis,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MeshcatVisualizer,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    Quaternion,
    Rgba,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import RenderDiagram

if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Warmup 
Recall the two rules of spatial algebra we discussed in class
$${}^AX^B{}^BX^C = {}^AX^C$$
$$\big[{}^AX^B\big]^{-1} = {}^BX^A$$

In Drake, you can multiply frames by 
```
X_AB.multiply(X_BC)
X_AB @ X_BC
```
You may also invert a rigid transform via the inverse method
```
X_AB.inverse()
```
Now suppose you have 4 frames, namely, the world frame, frame A, frame B, and frame C defined as below.

-- frame A expressed in the world frame `(X_WA)`

-- frame B expressed in frame A `(X_AB)`

-- frame C expressed in frame C `(X_CB)`

**Calcuate the following transforms by filling your code below in the designated functions.**


(1) `X_WB`, frame B expressed in the world frame

(2) `X_CW`, the world frame expressed in frame C

In [3]:
def compute_X_WB(X_WA, X_AB, X_CB):
    """
    fill your code here
    """
    X_WB = RigidTransform()
    return X_WB

In [5]:
def compute_X_CW(X_WA, X_AB, X_CB):
    """
    fill your code here
    """
    X_CW = RigidTransform()
    return X_CW

In [7]:
# paste the output from the print statements into gradescope


X_WA = RigidTransform([0.1, 0.2, 0.3])
X_AB = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi / 2), np.array([-0.1, 0.4, 0.5])
)
X_CB = RigidTransform(
    RotationMatrix(np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]])),
    np.array([0.0, 0.0, 0.4]),
)

print(f"{compute_X_WB(X_WA, X_AB, X_CB).GetAsMatrix4()[2, 3]=}")
print(f"{compute_X_CW(X_WA, X_AB, X_CB).GetAsMatrix4()[0, 3]=}")

compute_X_WB(X_WA, X_AB, X_CB).GetAsMatrix4()[2, 3]=np.float64(0.8)
compute_X_CW(X_WA, X_AB, X_CB).GetAsMatrix4()[0, 3]=np.float64(0.6000000000000001)


# Design Grasp Pose

The grasp pose is commonly defined in the object frame so that the grasp pose ${^OX^G}$ is independent of the pose of the object. The grasp pose in the world frame can be computed by 

$${{^WX^G} = {}{^W}X^{O}} {^OX^G},$$


where $W$ stands for the world frame and $G$ denotes the grasp frame, following the convention in the textbook.


You should notice from the visualization below that the gripper frame is different from the world frame. In particular, the +y axis of the gripper frame points vertically downward, and the +z axis of the gripper points backward. This is an important observation for this exercise.

Simply commanding the gripper to the grasp pose does not ensure a collision-free path. To address this, we define a "pre-pick" pose—a nearby position with enough clearance to let the robot move into the grasp pose without collisions.

**Now for your exercise, use the gripper and object's orientation from the image to design a grasp pose that satisfy the conditions below**

- **gripper's position should be 0.25 meters above the target object in the world frame.**
- **gripper's z axis should align rotationally with object's x axis. the gripper should be 0.1 meters in front of the object with respect to the object's x axis.**
- **gripper's x axis should align rotationally with object's z axis**
- **write grasp pose in the object frame and the world frame**

**Once you have the grasp pose, also define a pre-pick pose that is 0.1 meters above the grasp pose in the world frame.**

**Remember that the X-axis is shown in red, Y-axis is in green, and Z-axis in blue.**

In [8]:
def design_grasp_pose(X_WO):
    """
    fill in our code below
    """
    X_OG = RigidTransform()
    X_WG = RigidTransform()
    X_WGprepick = RigidTransform()
    return X_OG, X_WG, X_WGprepick

# Design Goal Pose
We've figured out where the robot gripper needs to move given the initial pose of the objects. Now we need to decide where the gripper needs to go to place the object. 

You'll notice in the figure that the first initial is facing the right direction but is offset translationally (it is behind the robot). What we want is it to sit to the 0.4 meters left (-x in the world frame) of the second initial. 

The second initial is located at the right pose, but is rotated so that it is facing perpendicular to the robot. We want it to have the same orientation as the first initial. 

**Write a function that takes the pose of your first initial and second initial computes returns corrected poses for each. Then, given the grasp pose, find the correct gripper poses in the world frame to achieve these poses.**

In [10]:
def design_goal_poses(X_WO1, X_WO2, X_OG):
    """
    fill in our code below
    """
    X_WG1goal = RigidTransform()
    X_WG2goal = RigidTransform()
    return X_WG1goal, X_WG2goal

# From Keyframes to a Trajectory

Great! So to manipulate the two initials the robot is going to follow this trajectory
- Start at its default configuration (gripper opened)
- Move to the first prepick pose (gripper opened)
- Move to the grasp pose (gripper opened)
- Remain at the grasp pose (gripper closed)
- move to the default configuration (gripper closed)
- move to the first place pose (gripper closed)
- remain at the first place pose (gripper opened)
- return to the default configuration and repeat the process for the second initial


We've already solved for all of these poses! The next steps are to turn them into a valid trajectory we can command the robot to follow. 

**Your job is to implement the `MakeTrajectory` function.** It will take as input 
- a list of gripper poses of length L
- an 1xL numpy array of gripper states (distance between the two fingers)
- a list of timestamps of length L

It should return
- the **velocities** corresponding to the Piecewise trajectory defined in terms of gripper poses (of type `Trajectory`)
- a trajectory defining the state of the fingers.

In [12]:
def MakeTrajectory(X_Gs, finger_values, sample_times):
    robot_velocity_trajectory = None
    traj_wsg_command = None
    # TODO: define a PiecewisePose out of the X_Gs

    # TODO: set robot_velocity_trajectory to the derivative of the position trajectory you just defined

    # TODO: set traj_wsg_command to a PiecewisePolynomial that commands the fingers

    return robot_velocity_trajectory, traj_wsg_command

# Initializing our System

Let's spin up a system with all of our assets. First, we'll generate the asset files for our first and last initial as well as the table. Then, we define a DMD YAML file. Just like problem set 1! 
**Be sure to fill in your initials!**

In [14]:
initials = None

In [16]:
output_dir = Path("assets/")
for letter in initials:
    create_sdf_asset_from_letter(
        text=letter,
        font_name="DejaVu Sans",
        letter_height_meters=0.2,
        extrusion_depth_meters=0.04,
        output_dir=output_dir / f"{letter}_model",
        use_coacd=False,
        mass=0.1,
    )
letter_sdfs = [
    f"{Path.cwd()}/assets/{letter}_model/{letter}.sdf" for letter in initials
]

table_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="table">
    <pose>0 0 0 0 0 0</pose>
    <link name="table_link">
      <inertial>
        <mass>20</mass>
        <inertia>
          <ixx>1.0</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>1.0</iyy>
          <iyz>0.0</iyz>
          <izz>1.0</izz>
        </inertia>
      </inertial>
      <collision name="box_collision">
        <geometry>
          <box>
            <size>2 2 0.1</size>
          </box>
        </geometry>
      </collision>
      <visual name="box_visual">
        <geometry>
          <box>
            <size>2 2 0.1</size>
          </box>
        </geometry>
      </visual>
    </link>
  </model>
</sdf>
"""

table_sdf_path = "assets/table.sdf"
with open(table_sdf_path, "w") as f:
    f.write(table_sdf)

table_sdf = f"{Path.cwd()}/assets/table.sdf"
scenario_yaml = f"""directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
- add_model:
    name: table
    file: file://{table_sdf}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
        translation: [0.0, 0.0, -0.05]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: first_initial
    file: file://{letter_sdfs[0]}
    default_free_body_pose:
        {initials[0]}_body_link:
            translation: [0.6, 0.1, 0.01]
            rotation: !Rpy {{ deg: [90, 0, 0] }}
- add_model:
    name: last_initial
    file: file://{letter_sdfs[1]}
    default_free_body_pose:
        {initials[1]}_body_link:
            translation: [-0.2, -0.5, 0.01]
            rotation: !Rpy {{ deg: [90, 0, 90] }}

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

# A Jacobian Based Velocity Controller
Ok, we've specified the geometry of the scene, and our desired robot trajectory. Now we have to add the logic to relate our desired gripper velocities to robot joint velocities that we can command. 

**Your job is to implement the CalcOutput function. Recall that this takes the current context and sets a value for `output` based on this context.**

In [17]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        """
        fill in our code below.
        """
        # evaluate the V_G_port and q_port on the current context to get those values.

        # update the positions of the internal _plant_context according to `q`.
        # HINT: you can write to a plant context by calling `self._plant.SetPositions`

        # Compute the gripper jacobian
        # HINT: the jacobian is 6 x N, with N being the number of DOFs.
        # We only want the 6 x 7 submatrix corresponding to the IIWA

        # compute `v` by mapping the gripper velocity (from the V_G_port) to the joint space
        v = None
        output.SetFromVector(v)

# Putting everything together

We're almost done! Let's use the DMD we defined to initialize a diagram, and specify the keyframes as well!

We've added all the systems needed for the diagram to the builder. **Your job will be to connect all the necessary input and output ports for simulation.**

**Once you have everything working, take a video and upload it to gradescope!**

In [19]:
# load dmd and define station, builder, plant
with open("scenario.yaml", "w") as f:
    f.write(scenario_yaml)
scenario = LoadScenario(filename="scenario.yaml")
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder = DiagramBuilder()
builder.AddSystem(station)
plant = station.GetSubsystemByName("plant")

# get initial poses and build trajectory keyframes
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
X_WO1initial = plant.EvalBodyPoseInWorld(
    temp_plant_context, plant.GetBodyByName(f"{initials[0]}_body_link")
)
X_WO2initial = plant.EvalBodyPoseInWorld(
    temp_plant_context, plant.GetBodyByName(f"{initials[1]}_body_link")
)

X_OG, X_WG2pick, X_WG2prepick = design_grasp_pose(X_WO2initial)
_, X_WG1pick, X_WG1prepick = design_grasp_pose(X_WO1initial)
X_WG1goal, X_WG2goal = design_goal_poses(X_WO1initial, X_WO2initial, X_OG)

opened = 0.107
closed = 0.0
keyframes = [
    (X_WGinitial, opened),
    (X_WG2prepick, opened),
    (X_WG2pick, opened),
    (X_WG2pick, closed),
    (X_WGinitial, closed),
    (X_WG2goal, closed),
    (X_WG2goal, opened),
    (X_WGinitial, opened),
    (X_WG1prepick, opened),
    (X_WG1pick, opened),
    (X_WG1pick, closed),
    (X_WGinitial, closed),
    (X_WG1goal, closed),
    (X_WG1goal, opened),
    (X_WGinitial, opened),
]

gripper_poses = [keyframe[0] for keyframe in keyframes]
finger_states = np.asarray([keyframe[1] for keyframe in keyframes]).reshape(1, -1)
sample_times = [2 * i for i in range(len(gripper_poses))]
traj_V_G, traj_wsg_command = MakeTrajectory(gripper_poses, finger_states, sample_times)


V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(7))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# TODO: connect the joint velocity source to the pseudoinverse controller

# TODO: connect the controller to integrator to get joint angle commands

# TODO: connect the joint angles computed by the integrateor to the iiwa.position port on the manipulation station

# TODO: connect the "iiwa.position_measured" port on the station back to the relevant input port on the controller

# TODO: connect the wsg_source to the "wsg.position" input port of the station

# visualize axes
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{initials[0]}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{initials[1]}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
)

# build diagram, initialize integrator with initial joint angles
diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant.GetMyContextFromRoot(context),
        plant.GetModelInstanceByName("iiwa"),
    ),
)
diagram.ForcedPublish(context)
print(f"sanity check, simulation will run for {traj_V_G.end_time()} seconds")

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

sanity check, simulation will run for 28.0 seconds


RuntimeError: InputPort::Eval(): required InputPort[0] (u0) of System ::_::station::iiwa::velocity_interpolator::drake/systems/Multiplexer@000000001a04b480 (Multiplexer<double>) is not connected